<a href="https://colab.research.google.com/github/jzphlp/clab/blob/dev-clab/incremental-ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import math
import numpy as np
import pandas as pd
import dask.dataframe as dd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
from pprint import pprint
import lightgbm as lgb
from params_lgb import *
from df_utils import *

# make this into a function

In [ ]:
max_depth=7
num_leaves = (2 ** max_depth) - 50
num_leaves

78

In [ ]:
########################################################################################
# INPUTS
########################################################################################
save_dir = r"zdir"
parquets = r"parquets"
tcolx = 'pdem'
fcolx = ['cdem_dem','edem_dem','egm08','tdem_dem']
interval = 100#500#4500#

modelpath_i1 = os.path.join(save_dir,'model_i1.txt')
modelpath_i1_best = os.path.join(save_dir,'model_i1_best.txt')
np.random.seed(seed=seed)

params = params1


In [ ]:
########################################################################################
# MODELLING
########################################################################################
os.makedirs(save_dir, exist_ok=True)
filenames = glob.glob(f'{parquets}/*.parquet'); print(len(filenames))
filenames_shuffled_all = list(shuffle(filenames,random_state=seed))

loops = math.ceil(len(filenames_shuffled_all)/interval)
filenames_shuffled = filenames_shuffled_all[:interval]
train_filenames, val_filenames = train_test_split(
    filenames_shuffled,test_size=0.3, random_state=seed)
out_val = val_filenames


ddf_train = pd.read_parquet(train_filenames)
ddf_train = filter_values(ddf_train, tcolx, -20, 1000, -9999)
xtrain, ytrain = ddf_train.drop(tcolx, axis=1), ddf_train[tcolx]
del ddf_train

ddf_valid = pd.read_parquet(val_filenames)
ddf_valid = filter_values(ddf_valid, tcolx, -20, 1000, -9999)
xvalid, yvalid = ddf_valid.drop(tcolx, axis=1), ddf_valid[tcolx]
del ddf_valid
 # is there a way of spling x,y without having to load the data and then delete
train_ds = lgb.Dataset(data=xtrain,label=ytrain,feature_name=fcolx)
valid_ds = lgb.Dataset(data=xvalid,label=yvalid,feature_name=fcolx)

gbm = lgb.train(params,
                train_ds,
                num_round,
                valid_sets=valid_ds,
                feval=mean_error,
                keep_training_booster=True, # lear what it does
                callbacks=[lgb.early_stopping(stopping_rounds=50)])

gbm.save_model(modelpath_i1)
gbm.save_model(modelpath_i1_best,num_iteration=gbm.best_iteration)


pvalid = gbm.predict(xvalid)
derror = Rmetrics(yvalid, pvalid)
derror.to_csv(os.path.join(save_dir,'metrics_i1.csv'), index=False)
pprint(derror)

booster = None
for i in range(1,loops):
    print(f'Icrementtal at looop {i}/{loops}')
    start_index = interval*i
    if i == loops - 1:
        end_index = len(filenames_shuffled_all) - 1
    else:
        end_index = interval*i+interval
    filenames_shuffled = filenames_shuffled_all[start_index:end_index]
    train_filenames, val_filenames = train_test_split(
        filenames_shuffled,test_size=0.3, random_state=seed)
    out_val = np.append(out_val,val_filenames)

    ddf_train = pd.read_parquet(train_filenames)
    ddf_train = filter_values(ddf_train, tcolx, -20, 1000, -9999)
    xtrain, ytrain = ddf_train.drop(tcolx, axis=1), ddf_train[tcolx]
    del ddf_train

    ddf_valid = pd.read_parquet(val_filenames)
    ddf_valid = filter_values(ddf_valid, tcolx, -20, 1000, -9999)
    xvalid, yvalid = ddf_valid.drop(tcolx, axis=1), ddf_valid[tcolx]
    del ddf_valid

    # is there a way of spling x,y without having to load the data and then delete
    train_ds = lgb.Dataset(data=xtrain,label=ytrain,feature_name=fcolx)
    valid_ds = lgb.Dataset(data=xvalid,label=yvalid,feature_name=fcolx)

    keep_training_booster_val = True
    booster = None

   # print(f'INTITIAL############')
    initial_scores = gbm.predict(xtrain) #####INITIAL SCORES
    #derror = Rmetrics(ytrain, initial_scores)
    #pprint(derror)
    #gbm.add_valid(train_ds, initial_scores)
    #gbm.continue_training(init_scores=initial_scores)
    train_ds.set_init_score(initial_scores)
    gbm = lgb.train(params,
                train_ds,
                num_round,
                valid_sets=valid_ds,
                feval=mean_error,
                #init_model=booster,
                keep_training_booster=True, # lear what it does
                init_model=gbm,
                callbacks=[lgb.early_stopping(stopping_rounds=50)],
                )

    gbm.save_model(modelpath_i1)
    gbm.save_model(modelpath_i1_best,num_iteration=gbm.best_iteration)

out_val_pd = pd.DataFrame(out_val,columns=['files'])
out_val_pd.to_csv(os.path.join(save_dir,'validation_files.csv'),index=None)

pvalid = gbm.predict(xvalid)
derrorf = Rmetrics(yvalid, pvalid)
derrorf.to_csv(os.path.join(save_dir,'metrics_f1.csv'), index=False)
pprint(derror)
pprint(derrorf)


1049
Training until validation scores don't improve for 50 rounds
         R2      RSME       MAE  MedianAE      Bias
0  0.570582  0.995628  0.713177   0.51097  0.020527
Icrementtal at looop 1/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 2/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 3/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 4/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 5/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 6/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 7/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 8/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 9/11
Training until validation scores don't improve for 50 rounds
Icrementtal at looop 10/11
Training until valid

In [ ]:
# now its more like it

##### Assume you already have your existing model and data
existing_model = lgb.Booster(model_file='existing_model.txt')
new_data = np.random.rand(100, 10)  # Example new data, replace with your actual new data

##### Assume you have initial scores for the new data
initial_scores = np.random.rand(100)

##### Perform continued training with the new data and initial scores
existing_model.add_valid(new_data, initial_scores)
existing_model.continue_training(init_scores=initial_scores)

##### Save the updated model
existing_model.save_model('updated_model.txt')